# Query Expansion - letting LLMs generate multiple queries for the retriever

In [ ]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
# Load and index a blog post
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)

# VectorDB
embedding_model = AzureOpenAIEmbeddings(azure_deployment="text-embedding-ada-002", openai_api_version="2023-05-15")
vector_store = Chroma.from_documents(documents=splits, embedding=embedding_model)

## Retrieving using the MultiQueryRetriever

In [ ]:
import logging
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import AzureChatOpenAI


llm = AzureChatOpenAI(deployment_name="gpt-35-turbo", api_version="2023-03-15-preview", temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=vector_store.as_retriever(), llm=llm)

# setting up logging so we can extract the actual queries
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
question = "What are the approaches to Task Decomposition?"
unique_docs = retriever_from_llm.invoke(question)

In [ ]:
question = "Are there any vegetarian options on the menu?"
unique_docs = retriever_from_llm.invoke(question)